In [86]:
import requests, json
from bs4 import BeautifulSoup
import pandas as pd

site = requests.get('https://www.leedsgrandmosque.com/')
soup = BeautifulSoup(site.content, 'html.parser')

prayer_section = soup.find(class_="prayers")

prayer = [pr.get_text().capitalize() for pr in prayer_section.select(".row .prayer-name")]
ptime = [t.get_text() for t in prayer_section.select(".row .date")]
jammah = [j.get_text() for j in prayer_section.select(".row .jammah-date")]

info ={
    "Prayer": prayer,
    "Time": ptime,
    "Jammah": jammah,
}

pd.DataFrame(info)

,Prayer,Time,Jammah
0,Fajr,5:39,6:00
1,Shurooq,7:11,
2,Dhuhr,12:55,13:30
3,Asr,15:53,16:30
4,Maghrib,18:38,18:43
5,Isha,20:12,20:30


In [91]:
from datetime import datetime, date
import json

token = "secret_WHlov0Kr74x6RfOpTPNKVqKpDmp0qVZr0Lrn5Q1DMtt"
databaseId = "663d4598216844d776374ea2d8f42c46"

headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

def readDatabase(databaseId, headers):
    readUrl = f"https://api.notion.com/v1/databases/{databaseId}/query"

    res = requests.request("POST", readUrl, headers=headers)
    data = res.json()
    print(res.status_code)
    # print(res.text)

    with open('./Prayer_Time.json', 'w', encoding='utf8') as f:
        json.dump(data, f, ensure_ascii=False)

def createPage(databaseId, headers, title = ""):
    createUrl = 'https://api.notion.com/v1/pages'

    newPageData = {
        "parent": { "database_id": databaseId },
        "properties": {
            "Prayer": {
                "title": [
                    {
                        "text": {
                            "content": title
                        }
                    }
                ]
            },
            "Time": {
                "date": {
                    "start": str(datetime.strptime(str(date.today()) + " " + ptime[0], '%Y-%m-%d %H:%M')),
                    #"end": "2021-05-17T12:00:00Z"
                    "time_zone": "Europe/London"
                }
            },
            "Jummah": {
                "date": {
                    "start": str(datetime.strptime(str(date.today()) + " " + jammah[0], '%Y-%m-%d %H:%M')),
                    #"end": "2021-05-17T12:00:00Z"
                    "time_zone": "Europe/London"
                }
            }
        }
    }
    
    data = json.dumps(newPageData)
    # print(str(uploadData))

    res = requests.request("POST", createUrl, headers=headers, data=data)

    print(res.status_code)
    print(res.text)

def updatePage(headers):
    data = json.load(open("Prayer_Time.json"))

    page = dict.fromkeys(prayer)
    for i in data["results"]:
        page[i["properties"]["Prayer"]["title"][0]["plain_text"]] = i["id"]
    info["pageId"] = list(page.values())

    for i in range(len(page)):
        p = info["pageId"][i]
        updateUrl = f"https://api.notion.com/v1/pages/{p}"

        try:
            day = {
                "start": str(datetime.strptime(str(date.today()) + " " + jammah[i], '%Y-%m-%d %H:%M')),
                #"end": "2021-05-17T12:00:00Z"
                "time_zone": "Europe/London"
            }
        except:
            day = None

        updateData = {
            "properties": {
                "Time": {
                    "date": {
                        "start": str(datetime.strptime(str(date.today()) + " " + ptime[i], '%Y-%m-%d %H:%M')),
                        #"end": "2021-05-17T12:00:00Z"
                        "time_zone": "Europe/London"
                    }
                },
                "Jummah": {
                    "date": day
                }
            }
        }

        data = json.dumps(updateData)

        response = requests.request("PATCH", updateUrl, headers=headers, data=data)

    print(response.status_code)
    print(response.text)

# readDatabase(databaseId, headers)
# createPage(databaseId, headers, "Fajr")
updatePage(headers)


200
{"object":"page","id":"bcabb244-f0bb-401a-9db0-0a07f52266d5","created_time":"2022-06-09T16:26:00.000Z","last_edited_time":"2022-10-04T13:44:00.000Z","created_by":{"object":"user","id":"a0d7d6f1-4611-4b95-afb3-6cc503a70ad6"},"last_edited_by":{"object":"user","id":"2498057d-edaf-4953-b319-8da7de239862"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"663d4598-2148-44c7-9639-4eb2d8f42c46"},"archived":false,"properties":{"Time":{"id":"NXCn","type":"date","date":{"start":"2022-10-04T20:12:00.000+01:00","end":null,"time_zone":null}},"Jummah":{"id":"qNie","type":"date","date":{"start":"2022-10-04T20:30:00.000+01:00","end":null,"time_zone":null}},"Prayer":{"id":"title","type":"title","title":[{"type":"text","text":{"content":"Isha","link":null},"annotations":{"bold":false,"italic":false,"strikethrough":false,"underline":false,"code":false,"color":"default"},"plain_text":"Isha","href":null}]}},"url":"https://www.notion.so/Isha-bcabb244f0bb401a9db00a07f52266d5"}
